In [1]:
%matplotlib inline
import sys
sys.path.append('/apps')
import django
django.setup()
from drivers.graph_models import TreeNode, Order, Family, graph, pickNode
from traversals.strategies import sumTrees, UniformRandomSampleForest

# Getting Started with Tensor Flow
Here I´m taking the tutorials from: https://www.tensorflow.org/get_started/get_started

In [2]:
## importation
import tensorflow as tf

## Create two nodes (computing nodes)
Each node receives a set of tensors and returns another set of tensors. In the case of constant nodes it receives the empty set as input.



In [3]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

(<tf.Tensor 'Const:0' shape=() dtype=float32>, <tf.Tensor 'Const_1:0' shape=() dtype=float32>)


The nodes are defined abstractically. For evaluating them we need to create (initialize) as *session*.

The following code creates a Session object and then invokes its run method to run enough of the computational graph to evaluate node1 and node2. By running the computational graph in a session as follows:



In [4]:
session = tf.Session()
print(session.run([node1,node2]))

[3.0, 4.0]


We can perform algebraic computations. For example, sum node1 and node2

In [5]:
node3 = tf.add(node1,node2)
print("node 3 is:",node3)
print("session.run(node3):",session.run(node3))

('node 3 is:', <tf.Tensor 'Add:0' shape=() dtype=float32>)
('session.run(node3):', 7.0)


In [10]:
type(node1)
print("node1 %s , node3 %s"%(type(node1),type(node3)))

node1 <class 'tensorflow.python.framework.ops.Tensor'> , node3 <class 'tensorflow.python.framework.ops.Tensor'>


### Placeholder
Placeholder is a type of variable that doesn´t have a defined value. it is a promise to provide a value later.

In [11]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b # Shortcut for tf.add

In [12]:
type(a)

tensorflow.python.framework.ops.Tensor

*The preceding three lines are a bit like a function or a lambda in which we define two input parameters (a and b) and then an operation on them. We can evaluate this graph with multiple inputs by using the feed_dict argument to the run method to feed concrete values to the placeholders:*

In [10]:
session.run(adder_node,{a:3,b:4.5})

7.5

In [14]:
session.run(adder_node, {a:[1,2,3,4] , b : [1,1,1,1]})

array([ 2.,  3.,  4.,  5.], dtype=float32)

Eventhough the adder_node is made of *placeholders* we can still add it a any node

In [16]:
add_and_triple = adder_node * 3.
print(session.run(add_and_triple, {a: 3, b: 4.5}))

22.5


In [14]:
## Variables
W = tf.Variable([.3],dtype=tf.float32)
b = tf.Variable([-.3],dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

$y = x^t W + b $ 

In [15]:
type(W)

tensorflow.python.ops.variables.Variable

<bound method Graph.device of <tensorflow.python.framework.ops.Graph object at 0x7fcd1689ea10>>

For initializing all the variables in tensorflow we need to use the function init

In [18]:
initializer = tf.global_variables_initializer()
session.run(initializer)

In [19]:
session.run(linear_model, {x: [0.3, 20, 36.4454, 4.33
                            ]})

array([ -0.21000001,   5.69999981,  10.63362026,   0.99900001], dtype=float32)

## Training
A loss function measures how far apart the current model is from the provided data. We'll use a standard loss model for linear regression, which sums the squares of the deltas between the current model and the provided data. linear_model - y creates a vector where each element is the corresponding example's error delta. We call tf.square to square that error. Then, we sum all the squared errors to create a single scalar that abstracts the error of all examples using tf.reduce_sum

In [20]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(session.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


In [26]:
type(linear_model)

tensorflow.python.framework.ops.Tensor

In [27]:
tf.reduce_sum?

## gradient descend
We´ll use the symbolic computation in TensorFlow to calculate the derivative (gradient) of the linear model.

In [32]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
type(train)

tensorflow.python.framework.ops.Operation

In [22]:
# Reset values
session.run(initializer)

In [34]:
K = tf.assign(W,[-1])
G = tf.assign(b,[0])
session.run([K,G])
for i in range(100):
    session.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
    print(session.run([W,b]))


[array([-0.80000001], dtype=float32), array([ 0.08], dtype=float32)]
[array([-0.736], dtype=float32), array([ 0.1136], dtype=float32)]
[array([-0.71711999], dtype=float32), array([ 0.131712], dtype=float32)]
[array([-0.71319038], dtype=float32), array([ 0.14459905], dtype=float32)]
[array([-0.71419597], dtype=float32), array([ 0.1556692], dtype=float32)]
[array([-0.71681225], dtype=float32), array([ 0.16605486], dtype=float32)]
[array([-0.71993589], dtype=float32), array([ 0.17613292], dtype=float32)]
[array([-0.72320092], dtype=float32), array([ 0.18602946], dtype=float32)]
[array([-0.72648627], dtype=float32), array([ 0.1957873], dtype=float32)]
[array([-0.72975194], dtype=float32), array([ 0.20542157], dtype=float32)]
[array([-0.73298508], dtype=float32), array([ 0.21493822], dtype=float32)]
[array([-0.73618168], dtype=float32), array([ 0.22434019], dtype=float32)]
[array([-0.73934072], dtype=float32), array([ 0.2336293], dtype=float32)]
[array([-0.74246216], dtype=float32), array([

### Let´s see the results!


In [41]:
session.run([W,b])
session.run([loss],{ x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

[0.048223704]

In [38]:
## The full script
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11
